In [17]:
import warnings
import pickle
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [38]:
PATH_TO_DATA = '/home/andrei/Desktop/alice_kaggle'
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')
train_df.sort_values('time1')
#train_df = train_df.loc[:10000]
#test_df = test_df.loc[:10000]
y = train_df['target']
train_df.drop('target', axis=1, inplace=True)
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11


In [39]:
# загрузим словарик сайтов
with open(r"/home/andrei/Desktop/alice_kaggle/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])

text_columns = ['site_column%s' % i for i in range(1, 11)]
train_df['text_col'] = ''
test_df['text_col'] = ''

for i in range(1, 11):
    site_c = 'site{}'.format(i)
    site_name_c = 'site_column{}'.format(i)
    train_df[site_name_c] = sites_dict_df.loc[train_df[site_c]].values
    test_df[site_name_c] = sites_dict_df.loc[test_df[site_c]].values
    train_df['text_col'] += train_df[site_name_c]
    test_df['text_col'] += test_df[site_name_c]
    
print(train_df.head())

all_train_text = pd.concat([train_df['site_column{}'.format(i)].astype('U') for i in range(1, 11)])  
char_vec = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), max_features=100000)
word_vec = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_features=100000)
# fix wordofvec
word_vec.fit(all_train_text.astype('U'))

всего сайтов: 48371
            site1               time1  site2               time2    site3  \
session_id                                                                  
1             718 2014-02-20 10:02:45    NaN                 NaT      NaN   
2             890 2014-02-22 11:19:50  941.0 2014-02-22 11:19:50   3847.0   
3           14769 2013-12-16 16:40:17   39.0 2013-12-16 16:40:18  14768.0   
4             782 2014-03-28 10:52:12  782.0 2014-03-28 10:52:42    782.0   
5              22 2014-02-28 10:53:05  177.0 2014-02-28 10:55:22    175.0   

                         time3    site4               time4  site5  \
session_id                                                           
1                          NaT      NaN                 NaT    NaN   
2          2014-02-22 11:19:51    941.0 2014-02-22 11:19:51  942.0   
3          2013-12-16 16:40:19  14769.0 2013-12-16 16:40:19   37.0   
4          2014-03-28 10:53:12    782.0 2014-03-28 10:53:42  782.0   
5          2014-02-2

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [40]:
X_tfidf = word_vec.transform(pd.concat((train_df['text_col'], test_df['text_col'])).values.astype('U'))

In [41]:
alice_common = dict()
for i in range(1, 11):
    a = train_df[y == 0]['site_column%s' % i].value_counts()
    for k, v in dict(a).items():
        alice_common.setdefault(k, 0)
        alice_common[k] += v
alice_common = list(alice_common.items())
alice_common.sort(key=lambda x: x[1], reverse=True)
alice_common = set([_[0] for _ in alice_common[:30]])


def get_part_from_hour(h):
    if h < 6:
        return 0
    if h < 12:
        return 1
    if h < 18:
        return 2
    return 3


def get_session_duration(row):
    time_values = row[['time%s' % i for i in range(1, 10)]].dropna().values
    duration = (time_values.max() - time_values.min()).total_seconds()
    return duration


def get_count_of_good_sites(row):
    res = 0
    for i in range(1, 11):
        res += row['site_column%s' % i] in alice_common
    return res


def add_extra_features(df):
    df['hour'] = df['time1'].dt.hour
    
    for i in range(2, 4):
        df['delta%s' % (i-1)] = (df['time%s' % i] - df['time%s' % (i-1)]).dt.total_seconds()

    #df['duration'] = df.apply(get_session_duration, axis=1)
    
    df['count_good'] = df.apply(get_count_of_good_sites, axis=1)
    
    return df


def get_scaled_features(df):
    scalable_columns = ['hour', 'delta1',  'count_good'] # 'duration']
    return MinMaxScaler().fit_transform(df[scalable_columns].fillna(-1))


def get_part_of_day(df):
    df['part_of_day'] = np.array(list(map(lambda v: get_part_from_hour(v), df['time1'].dt.hour)))
    return OneHotEncoder().fit_transform(df[['part_of_day']])


def get_work_time(df):
    df['hour'] = df['time1'].dt.hour
    df['is_work_time'] = df['hour'].apply(lambda x: 8 <= x <= 17)
    return OneHotEncoder().fit_transform(df[['is_work_time']])


def get_extra_features(df):
    return get_scaled_features(add_extra_features(df))

In [48]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    idx = int(round(X.shape[0] * ratio))
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [45]:
# sequence of indices
sites_flatten = full_sites.values.flatten()

# and the matrix we are looking for
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0]  + 10, 10)))[:, 1:]

In [44]:
full_df = pd.concat([train_df, test_df])
sites = ['site%s' % i for i in range(1, 11)]
# Index to split the training and test data sets
idx_split = train_df.shape[0]
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0


In [ ]:
%%time
_train = pd.concat((train_df, test_df))
#X_train = full_sites_sparse[:idx_split, :]
print(full_sites_sparse.shape)
print(X_tfidf.shape)
print(_train.shape)
X_train = hstack([full_sites_sparse,# X_tfidf,
                  get_part_of_day(_train), 
                  get_work_time(_train),
                  get_extra_features(_train)
                 ], format='csr')


(336358, 48371)
(336358, 100000)
(336358, 31)


In [49]:
# Calculate metric on the validation set
print(get_auc_lr_valid(X_train[:idx_split], y))

0.979840797432511


In [50]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [51]:
lr = LogisticRegression(C=1, random_state=17, n_jobs=-1).fit(X_train[:idx_split], y)
X_test = X_train[idx_split:,:]
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'alice.csv')